#**Sequential Model**
We will execute the steps to define a sequential model that can predict a caption for the image when given image encodings and word embeddings as an input.

##Importing Libraries

In [ ]:
from keras.models import Input, Model
from keras.layers import Embedding, Dense, LSTM, Dropout, BatchNormalization, add
from keras.utils import plot_model
from pickle import load, dump 

In [ ]:
#Declaring global variables
max_length = 39
dict_size = 2355
target_size = (299, 299, 3)
embedding_size = 300

##Sequential Model Architecture
The model consists of layers that can process image encodings as well as word embeddings. We define a dropout layer and dense layer for image encodings. We define a dropout layer and LSTM layer for word embeddings. After that we add the 2 inputs and define a dense layer on it with 'relu' activation. Finally, we apply a sigmoid layer as the output layer to generate the prediction.

In [ ]:
#Processing image encodings
input1 = Input(shape = (2048,))
X1 = Dropout(0.5)(input1)
X1 = Dense(300, activation = 'relu')(X1)

#Processing word embeddings
input2 = Input(shape = (max_length,))
X2 = Embedding(dict_size, embedding_size, mask_zero = True)(input2)
X2 = Dropout(0.5)(X2)
X2 = LSTM(300)(X2)

decoder = add([X1, X2])
decoder = Dense(300, activation = 'relu')(decoder)
outputs = Dense(dict_size, activation = 'softmax')(decoder)

In [ ]:
model = Model(inputs = [input1, input2], outputs = outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 39)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 39, 300)      706500      input_9[0][0]                    
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 2048)         0           input_8[0][0]                    
____________________________________________________________________________________________

In [ ]:
with open("embedding_matrix.pkl", "rb") as f:
  embedding_matrix = load(f)

In [ ]:
#Setting the weights of the embeeding layer as the embedding matrix generated in the pre-processing phase and freezing them
print(model.layers[2])
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
summary = model.summary()
print(summary)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 39)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 39, 300)      706500      input_9[0][0]                    
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 2048)         0           input_8[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

In [ ]:
#Saving the sequential model as a json file and saving the weigths of the model as '.h5' file so that we can use them when we finally train the sequential model
model_json = model.to_json()
with open("model.json", "w") as f:
  f.write(model_json)
model.save_weights("model.h5")